In [1]:
from utils import read_from_jsonl
train_data = read_from_jsonl("train.jsonl")
test_data = read_from_jsonl("test.jsonl")

In [2]:
train_data[0]

{'tokens': ['EU',
  'rejects',
  'German',
  'call',
  'to',
  'boycott',
  'British',
  'lamb',
  '.'],
 'chunk_tags': [1, 1, 1, 0, 1, 1, 1, 0, 1],
 'pos_tags': [1, 4, 3, 1, 4, 4, 3, 1, 4]}

In [3]:
from copy import deepcopy
train_data_encoded = deepcopy(train_data)
test_data_encoded = deepcopy(test_data)

In [4]:
import numpy as np

curr_one_hot_encoding_mapping = {
    1: [1, 0, 0, 0],
    2: [0, 1, 0, 0],
    3: [0, 0, 1, 0],
    4: [0, 0, 0, 1]
}

prev_one_hot_encoding_mapping = {
    # 0 for start of string sequence
    0: [1, 0, 0, 0, 0],
    1: [0, 1, 0, 0, 0],
    2: [0, 0, 1, 0, 0],
    3: [0, 0, 0, 1, 0],
    4: [0, 0, 0, 0, 1]
}

for i, data in enumerate(train_data):
    for j, pos in enumerate(data["pos_tags"]):
        if j == 0:
            prev = prev_one_hot_encoding_mapping[0]
            curr = curr_one_hot_encoding_mapping[pos]
        else:
            prev = prev_one_hot_encoding_mapping[data["pos_tags"][j-1]]
            curr = curr_one_hot_encoding_mapping[pos]
        
        train_data_encoded[i]["pos_tags"][j] = np.array(prev + curr)
        

In [5]:
train_data_encoded[0]


{'tokens': ['EU',
  'rejects',
  'German',
  'call',
  'to',
  'boycott',
  'British',
  'lamb',
  '.'],
 'chunk_tags': [1, 1, 1, 0, 1, 1, 1, 0, 1],
 'pos_tags': [array([1, 0, 0, 0, 0, 1, 0, 0, 0]),
  array([0, 1, 0, 0, 0, 0, 0, 0, 1]),
  array([0, 0, 0, 0, 1, 0, 0, 1, 0]),
  array([0, 0, 0, 1, 0, 1, 0, 0, 0]),
  array([0, 1, 0, 0, 0, 0, 0, 0, 1]),
  array([0, 0, 0, 0, 1, 0, 0, 0, 1]),
  array([0, 0, 0, 0, 1, 0, 0, 1, 0]),
  array([0, 0, 0, 1, 0, 1, 0, 0, 0]),
  array([0, 1, 0, 0, 0, 0, 0, 0, 1])]}

In [6]:
for i in train_data_encoded:
    chunk_tags = np.array(i["chunk_tags"])
    i["chunk_tags"] = chunk_tags
    pos_tags = np.array(i["pos_tags"])
    pos_tags.reshape(1, pos_tags.shape[0], pos_tags.shape[1])
    i["pos_tags"] = pos_tags


In [7]:
train_data_encoded[0]

{'tokens': ['EU',
  'rejects',
  'German',
  'call',
  'to',
  'boycott',
  'British',
  'lamb',
  '.'],
 'chunk_tags': array([1, 1, 1, 0, 1, 1, 1, 0, 1]),
 'pos_tags': array([[1, 0, 0, 0, 0, 1, 0, 0, 0],
        [0, 1, 0, 0, 0, 0, 0, 0, 1],
        [0, 0, 0, 0, 1, 0, 0, 1, 0],
        [0, 0, 0, 1, 0, 1, 0, 0, 0],
        [0, 1, 0, 0, 0, 0, 0, 0, 1],
        [0, 0, 0, 0, 1, 0, 0, 0, 1],
        [0, 0, 0, 0, 1, 0, 0, 1, 0],
        [0, 0, 0, 1, 0, 1, 0, 0, 0],
        [0, 1, 0, 0, 0, 0, 0, 0, 1]])}

In [8]:
for i in range(5):
    print(train_data_encoded[i]["pos_tags"].shape)
    print(train_data_encoded[i]["chunk_tags"].shape)

(9, 9)
(9,)
(2, 9)
(2,)
(2, 9)
(2,)
(30, 9)
(30,)
(31, 9)
(31,)


## Dataloader implementing Above things is in utils.py

In [9]:
from utils import DataLoader
train_data = read_from_jsonl("train.jsonl")
test_data = read_from_jsonl("test.jsonl")

train_data_loader = DataLoader(train_data, batch_size=32)
test_data_loader = DataLoader(test_data, batch_size=32)

In [10]:
from utils import RecurrentPerceptron

a = RecurrentPerceptron()
a.train_from_dataloader(train_data_loader, 0.05, 100, debug=True)

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]


DEBUG LOGS FOR STEP
------------------------------------------------------------
x_shape (9, 9)
y_shape torch.Size([9])
y_p shape torch.Size([9])
y shape torch.Size([9])
y tensor([1., 1., 1., 0., 1., 1., 1., 0., 1.])
y_p tensor([0.4852, 0.2586, 0.5729, 0.3274, 0.2876, 0.6004, 0.4944, 0.3436, 0.2845])
loss=tensor(0.7964), output : tensor([0.2845])
------------------------------------------------------------
DEBUG LOGS FOR STEP
------------------------------------------------------------
x_shape (2, 9)
y_shape torch.Size([2])
y_p shape torch.Size([2])
y shape torch.Size([2])
y tensor([1., 0.])
y_p tensor([0.5514, 0.2852])
loss=tensor(0.4656), output : tensor([0.2852])
------------------------------------------------------------
DEBUG LOGS FOR STEP
------------------------------------------------------------
x_shape (2, 9)
y_shape torch.Size([2])
y_p shape torch.Size([2])
y shape torch.Size([2])
y tensor([1., 0.])
y_p tensor([0.5517, 0.3437])
loss=tensor(0.5079), output : tensor([0.3437])

RuntimeError: all elements of input should be between 0 and 1